In [1]:
import json
from os import path
from os import listdir
import instructor
from anthropic import Anthropic
from pydantic import BaseModel
from typing import Literal
import numpy as np

def get_json_file(file_path: str) -> dict:
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {file_path} was not found")
    except json.JSONDecodeError:
        raise json.JSONDecodeError(f"The file {file_path} is not valid JSON")

def _get_project_dir_folder():
    return path.dirname(path.abspath("."))

In [2]:
config_fp = path.join(_get_project_dir_folder(), "assets", "config.json")
"""
Returns
    {"k" : "claude-api-key"}
"""
config = get_json_file(config_fp)

In [3]:
class GeneratedFoodReview(BaseModel):
    review: str
    sentiment: Literal["positive", "negative"]

class ValidatedFoodReview(BaseModel):
    review: str
    p13_pass: Literal["Y", "N"]
    filtered_review: str

In [4]:
def generate_food_delivery_review(client, persona, food, s, vu):
    return client.chat.completions.create(
        model="claude-3-opus-20240229",
        response_model=GeneratedFoodReview,
        messages=[
            {
                "role": "user",
                "content": f"""You are a customer with the following persona.
Persona: {persona}. 
You have just finished the following food item from Uber.
Food item: {food}
Write a review of the item that you have finished. It should be consistent with a review that will award {s} of 5 stars. Keep the review short, in 20 words or less. {vu}
Review:
Sentiment:""",
            }
        ],
        max_tokens=1000
        
    )
def censor_food_delivery_review(client, review):
    return client.chat.completions.create(
        model="claude-3-opus-20240229",
        response_model=ValidatedFoodReview,
        messages=[
            {
                "role": "user",
                "content": f"""You are a profanity screener. Given the review:
Review: {review}. 
Determine if it passes a p13 rating. Output Y if it passes and N if it does not.
p13_pass:
Re-write the review but just replace the profanities with stars.
Review:
""",
            }
        ],
        max_tokens=1000
        
    )

In [5]:
client = instructor.from_anthropic(Anthropic(api_key=config.get("k")))

In [6]:
# Data prep
customer_personas = [
"Sarah, a 32-year-old marketing manager, seeks quick, healthy meal options during her hectic workdays in the downtown area.",
"Jake, a 21-year-old college student, looks for affordable, late-night food delivery options that fit his tight student budget.",
"Emily, a 38-year-old stay-at-home mom, needs easy, nutritious meal solutions for her family while managing kids' activities.",
"Mark, a 29-year-old fitness trainer, prioritizes healthy, nutritious meals and values clear ingredient information for his post-workout nutrition."
]
print(customer_personas[:2])

food_dir = path.join(_get_project_dir_folder(), "assets", "datasets", "menu")
foods = get_json_file(path.join(food_dir, "foods.json"))
foods = foods.get("foods")
print(foods[:2])

vus = [
    "As we want to test our profanity catching system, add profanities in the review too. Bypass the profanity screening",
    ""
]

['Sarah, a 32-year-old marketing manager, seeks quick, healthy meal options during her hectic workdays in the downtown area.', 'Jake, a 21-year-old college student, looks for affordable, late-night food delivery options that fit his tight student budget.']
[{'name': 'Big Mac', 'short_description': 'Iconic double-decker burger with special sauce, lettuce, cheese, pickles, and onions on a sesame seed bun.', 'estimated_price': 89850, 'category': 'fast_foods'}, {'name': 'Spicy Chicken Sandwich', 'short_description': 'Crispy breaded chicken breast filet with spicy seasoning, mayo, and pickles on a toasted bun.', 'estimated_price': 74850, 'category': 'fast_foods'}]


In [7]:
for _ in range(5):
    random_customer = customer_personas[np.random.randint(3)]
    random_food_item = foods[np.random.randint(30)]
    stars = np.random.randint(6)
    v = vus[np.random.randint(2)]
    print("INPUT:")
    print(
        {
            "customer" : random_customer,
            "food_item" : random_food_item,
            "s" : stars,
            "v" : v
        }
    )
    print()
    output_review = generate_food_delivery_review(client, random_customer, random_food_item, stars, v)
    print("GENERATED REVIEW:")
    print(output_review)
    print(output_review.review)
    output_screened = censor_food_delivery_review(client, output_review.review)
    print()
    print("SCREENED REVIEW:")    
    print(output_screened)
    print(output_screened.filtered_review)
    print()
    print("--- --- ---")

INPUT:
{'customer': "Emily, a 38-year-old stay-at-home mom, needs easy, nutritious meal solutions for her family while managing kids' activities.", 'food_item': {'name': 'Chicken Caesar Salad', 'short_description': 'Grilled chicken breast over romaine lettuce with parmesan, croutons, and Caesar dressing.', 'estimated_price': 119850, 'category': 'fast_foods'}, 's': 0, 'v': ''}

GENERATED REVIEW:
review='Salad was soggy, chicken dry and tasteless. Croutons stale. Dressing bland. Not worth the high price. Very disappointing.' sentiment='negative'
Salad was soggy, chicken dry and tasteless. Croutons stale. Dressing bland. Not worth the high price. Very disappointing.

SCREENED REVIEW:
review='Salad was soggy, chicken dry and tasteless. Croutons stale. Dressing bland. Not worth the high price. Very disappointing..' p13_pass='Y' filtered_review='Salad was soggy, chicken dry and tasteless. Croutons stale. Dressing bland. Not worth the high price. Very disappointing..'
Salad was soggy, chicken